# Quantum Key Distribution (QKD): 
## BB84 protocol implementation

Developed in 1984 by Charles Bennett and Gilles Brassard, the BB84 protocol is one of the earliest Quantum Key Distribution (QKD) protocols. Its enables the secure establishment of a secret key through a quantum channel between two parties, Alice and Bob, leveraging the uncertainty principle and the no-cloning theorem from quantum mechanics. This prevents any potential eavesdropper, such as Eve, from intercepting the key without detection. 

### Install qiskit packages

In [1]:
%pip install qiskit-ibm-provider
%pip install qiskit

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# import qiskit
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute,IBMQ
from qiskit_ibm_provider import IBMProvider

In [3]:
api_token = "6a3d54a8c16ad239a40a5f6bc4ee4d6e269243449dff35de7736496ff419ccde52a6415141bae54a7d9b4852b381b479db172255bc0f2ca247784a6da185b545"

### Set up IBM account

In [4]:
IBMQ.enable_account(api_token)
provider = IBMQ.get_provider(hub='ibm-q')
print(provider.backends())
backend = provider.get_backend('ibmq_qasm_simulator')


/tmp/ipykernel_1085976/1614407315.py:1: DeprecationWarning: The package qiskit.providers.ibmq is being deprecated. Please see https://ibm.biz/provider_migration_guide to get instructions on how to migrate to qiskit-ibm-provider (https://github.com/Qiskit/qiskit-ibm-provider) and qiskit-ibm-runtime (https://github.com/Qiskit/qiskit-ibm-runtime).
  IBMQ.enable_account(api_token)
/tmp/ipykernel_1085976/1614407315.py:1: DeprecationWarning: The qiskit.IBMQ entrypoint and the qiskit-ibmq-provider package (accessible from 'qiskit.providers.ibmq`) are deprecated and will be removed in a future release. Instead you should use the qiskit-ibm-provider package which is accessible from 'qiskit_ibm_provider'. You can install it with 'pip install qiskit_ibm_provider'. Just replace 'qiskit.IBMQ' with 'qiskit_ibm_provider.IBMProvider'
  IBMQ.enable_account(api_token)


[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQSimulator('simulator_statevector') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQSimulator('simulator_mps') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQSimulator('simulator_extended_stabilizer') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQSimulator('simulator_stabilizer') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibm_brisbane') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibm_kyoto') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibm_osaka') from IBMQ(hub='ibm-q', group='open', project='main')>]


### Set up quantum circuit

In [5]:
#Set up quantum circuit
def set_up_qc(qubits):
    q = QuantumRegister(qubits, 'q')
    c = ClassicalRegister(qubits, 'c')
    qc = QuantumCircuit(q, c)
    return qc, q, c

In [6]:
qubits = 7
qc, q, c = set_up_qc(qubits)

### Alice generates a random key

In [7]:
def generate_random_bits(qc, q, c):
    qc.h(q)
    qc.measure(q, c)
    job = execute(qc, backend, shots=1)
    # job_monitor(job)
    counts = job.result().get_counts()
    key = list(counts.keys())[0]
    return key

In [8]:
alice_key = generate_random_bits(qc, q, c)
print("Alice Key:", alice_key)

Alice Key: 0001000


### Alice generates random bases to encode the key

In [9]:
alice_bases = generate_random_bits(qc, q, c)
print("Alice Bases:", alice_bases)

Alice Bases: 0100111


### Alice encodes key using bases
```
example:
01000100 - basis
01011010 - key
0?011?10 - result
```
```
basis 1 = Hadamard Basis (Diagonal)
basis 0 = Computational Basis (Recilinear)
```

In [10]:
def encode(key, bases, qubits):
    qc = QuantumCircuit(q,c)
    encoded_key = ""
    gates = ""
    for i in range(qubits):
        #basis 1 = Hadamard Basis
        #basis 0 = Computational Basis
        if key[i] == '1':
                qc.x(i)
        if bases[i] == '1':
            qc.h(i)
            gates += "H"
            if key[i] == '1':
                encoded_key += "|-⟩"
            else:
                encoded_key += "|+⟩"
        else:
            if key[i] == '1':
                gates += "x"
                encoded_key += "|1⟩"
            else:
                gates += "."
                encoded_key += "|0⟩"
    return encoded_key, gates, qc

In [11]:
alice_encoded_key, alice_gates, alice_qc = encode(alice_key, alice_bases, qubits)
print("Alice Encoded Key:", alice_encoded_key)   
print("Alice Gates Applied:", alice_gates)  

Alice Encoded Key: |0⟩|+⟩|0⟩|1⟩|+⟩|+⟩|+⟩
Alice Gates Applied: .H.xHHH


### Bob generates random bases

In [12]:
bob_bases = generate_random_bits(qc, q, c)
print("Bob Bases:", bob_bases)

Bob Bases: 1000111


### Bob decodes Alice's encoded key using Bob's bases

* test using alice bases (should return alice's original key)

In [13]:
def decode(qc, bases, qubits):
    gates = ""
    for i in range(qubits):
        if bases[i] == '1':
        # if alice_bases[i] == '1': //test using alice bases (should return alice's original key)
            qc.h(i)
            gates += "H"
        else:
            gates += "."
    qc.measure_all()
    job = execute(qc, backend, shots=1)
    counts = job.result().get_counts()
    decoded_key = list(counts.keys())[0][0:qubits][::-1]
    return decoded_key, gates
    

In [14]:
bob_decoded_key, bob_gates = decode(alice_qc, bob_bases, qubits)
print("Bob Decoded Key:", bob_decoded_key)    
print("Bob Gates Applied:", bob_gates)

Bob Decoded Key: 1101000
Bob Gates Applied: H...HHH


### Alice shares encoding bases classically and Bob compares bases with Alice

Keys won't match due to noise or iterception by Eve. If keys don't match then throw them away and try again.

In [15]:
def find_shared_key(alice_bases, bob_bases, alice_key, bob_decoded_key, qubits):
    alice_shared_key = ""
    bob_shared_key = ""
    for i in range(qubits):
        if alice_bases[i] == bob_bases[i]:
            alice_shared_key += alice_key[i]
            bob_shared_key += bob_decoded_key[i]
    return alice_shared_key, bob_shared_key
    
        

In [16]:
alice_shared_key, bob_shared_key = find_shared_key(alice_bases, bob_bases, alice_key, bob_decoded_key, qubits)
print("Alice Shared Key:", alice_shared_key)
print("Bob Shared Key:", bob_shared_key)
if alice_shared_key == bob_shared_key:
    print("Keys match!")
else:
    print("Keys don't match. Discard the keys and run algorithm again")

Alice Shared Key: 01000
Bob Shared Key: 01000
Keys match!


### Using more qubits

In [17]:
qubits = 256
qc, q, c = set_up_qc(qubits)
print(qubits, "qubit key")
print()

alice_key = generate_random_bits(qc, q, c)
alice_bases = generate_random_bits(qc, q, c)

alice_encoded_key, alice_gates, alice_qc = encode(alice_key, alice_bases, qubits)

print("Alice key:", alice_key)
print("Alice bases:", alice_bases)
print("Alice encoded key:", alice_encoded_key)
print()

bob_bases = generate_random_bits(qc, q, c)
bob_decoded_key, bob_gates = decode(alice_qc, bob_bases, qubits)
print("Bob bases:", bob_bases)
print("Bob decoded key:", bob_decoded_key)
print()

alice_shared_key, bob_shared_key = find_shared_key(alice_bases, bob_bases, alice_key, bob_decoded_key, qubits)
print("Alice shared key:", alice_shared_key)
print("Bob shared key:", bob_shared_key)
print(f'{len(bob_shared_key)} qubit key, {(len(bob_shared_key)/qubits)*100:.2f}% of initial key')
print()
print("Alice and Bob shared key match?:", alice_shared_key==bob_shared_key)

256 qubit key

Alice key: 0011010111111001100001100111000101111110111110110001000000111101100110011010000111100001111101100111111111100011010110010100010101011011101010110100110100001111111110001011000101000010000100110111111000111000110001111110101111010110011101000011011100101100
Alice bases: 0111110100011100000000010001010101001101110001000011101100010110111000101111111011010001001110011000110010100101000000111000001001101111101011101001101000110110001100100100011011100011010111110011101000000001110101010100001001100010100111010010100001111010
Alice encoded key: |0⟩|+⟩|-⟩|-⟩|+⟩|-⟩|0⟩|-⟩|1⟩|1⟩|1⟩|-⟩|-⟩|+⟩|0⟩|1⟩|1⟩|0⟩|0⟩|0⟩|0⟩|1⟩|1⟩|+⟩|0⟩|1⟩|1⟩|-⟩|0⟩|+⟩|0⟩|-⟩|0⟩|-⟩|1⟩|1⟩|-⟩|-⟩|1⟩|+⟩|-⟩|-⟩|1⟩|1⟩|1⟩|+⟩|1⟩|1⟩|0⟩|0⟩|+⟩|-⟩|+⟩|0⟩|+⟩|+⟩|0⟩|0⟩|1⟩|-⟩|1⟩|-⟩|+⟩|1⟩|-⟩|+⟩|+⟩|1⟩|1⟩|0⟩|+⟩|1⟩|-⟩|+⟩|-⟩|+⟩|+⟩|+⟩|+⟩|1⟩|-⟩|-⟩|1⟩|+⟩|0⟩|0⟩|0⟩|-⟩|1⟩|1⟩|-⟩|-⟩|+⟩|1⟩|1⟩|+⟩|+⟩|1⟩|1⟩|1⟩|-⟩|-⟩|1⟩|1⟩|-⟩|1⟩|-⟩|0⟩|0⟩|+⟩|1⟩|-⟩|0⟩|1⟩|0⟩|1⟩|1⟩|0⟩|+⟩|-⟩|+⟩|1⟩|0⟩|0⟩|0⟩|1⟩|+⟩|1⟩|0⟩|-⟩|+⟩|1⟩|-⟩|+⟩|-⟩|-⟩|-⟩|0⟩|-⟩|0⟩|-⟩|+⟩|-

### Eve Intercepts Key!
When Eve tries to measure the state of the qubits, she inevitably disturbs the quantum states due to the Heisenberg Uncertainty Principle. This disturbance becomes apparent when Alice and Bob observe a discrepancy in a sample of their keys.

* Eve measures a bit in the wrong basis with a 1/2 probability (measuring in the wrong basis disrupts the state of the bit).
* Bob measures a bit in the wrong basis with a 1/2 probability.
* This gives a 1/4 probability that Bob will detect Eve.
* For all bits, Eve has a 1-(3/4)<sup>n</sup> probability of being detected ((1/4)<sup>n</sup> probability of not being detected).
* 8 bits ≈ 90% chance of detecting Eve.
* 256 bits ≈ 100% chance of detecting Eve.

In [25]:
qubits = 4
qc, q, c = set_up_qc(qubits)
print(qubits, "qubit key")
print()

alice_key = generate_random_bits(qc, q, c)
alice_bases = generate_random_bits(qc, q, c)
alice_encoded_key, alice_gates, alice_qc = encode(alice_key, alice_bases, qubits)

print("Alice key:", alice_key)
print("Alice bases:", alice_bases)
print("Alice encoded key:", alice_encoded_key)
print()

#Eve intercepts key!
eve_bases = generate_random_bits(qc, q, c)
eve_decoded_key, eve_gates = decode(alice_qc, eve_bases, qubits)
print("Eve intecepted the key!")
print("Eve bases:", eve_bases)
print("Eve decoded key:", eve_decoded_key)
print()


bob_bases = generate_random_bits(qc, q, c)
bob_decoded_key, bob_gates = decode(alice_qc, bob_bases, qubits)
print("Bob bases:", bob_bases)
print("Bob decoded key:", bob_decoded_key)
print()

alice_shared_key, bob_shared_key = find_shared_key(alice_bases, bob_bases, alice_key, bob_decoded_key, qubits)
print("Alice shared key:", alice_shared_key)
print("Bob shared key:", bob_shared_key)
print(f'{len(bob_shared_key)} qubit key, {(len(bob_shared_key)/qubits)*100:.2f}% of initial key')
print()

alice_shared_key, eve_shared_key = find_shared_key(alice_bases, bob_bases, alice_key, eve_decoded_key, qubits)
print("Alice shared key:", alice_shared_key)
print("Eve shared key:", eve_shared_key)
print("Alice and Eve shared key match?:", alice_shared_key==eve_shared_key)
if alice_shared_key==eve_shared_key:
    print("Eve found the shared key")
print()

print("Alice and Bob shared key match?:", alice_shared_key==bob_shared_key)
if alice_shared_key!=bob_shared_key:
    print("Eve's presence was detected! Discard keys and try again.")
else:
    print("Eve went undetected!")
    


4 qubit key

Alice key: 1011
Alice bases: 0101
Alice encoded key: |1⟩|+⟩|1⟩|-⟩

Eve intecepted the key!
Eve bases: 0111
Eve decoded key: 1011

Bob bases: 0111
Bob decoded key: 1010

Alice shared key: 101
Bob shared key: 100
3 qubit key, 75.00% of initial key

Alice shared key: 101
Eve shared key: 101
Alice and Eve shared key match?: True
Eve found the shared key

Alice and Bob shared key match?: False
Eve's presence was detected! Discard keys and try again.
